# Building an Image Classifier and Integrating it into Web Applications Using AI Squared

In this demo, we are going to create an image classification model using TensorFlow and BeyondML and integrate it into an `.air` file to be consumed using the AI Squared Platform and Browser Extension

To do this, we are first going to install the `aisquared` and `beyondml` packages from `PyPI`, then we are going to use the `CIFAR10` dataset from TensorFlow to build an image classification model.

In [1]:
! pip install aisquared
! pip install beyondml

In [2]:
import tensorflow as tf
from beyondml import tflow
import aisquared
import numpy as np
from sklearn.metrics import classification_report, confusion_matrix

In [3]:
(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()

x_train = x_train/255
x_test = x_test/255

In [4]:
input_layer = tf.keras.layers.Input(x_train.shape[1:])
x = tflow.layers.MaskedConv2D(16, activation = 'relu', padding = 'same')(input_layer)
x = tf.keras.layers.BatchNormalization()(x)
x = tflow.layers.MaskedConv2D(16, activation = 'relu', padding = 'same')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.MaxPool2D()(x)
x = tflow.layers.MaskedConv2D(32, activation = 'relu', padding = 'same')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tflow.layers.MaskedConv2D(32, activation = 'relu', padding = 'same')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.MaxPool2D()(x)
x = tflow.layers.MaskedConv2D(64, activation = 'relu', padding = 'same')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tflow.layers.MaskedConv2D(64, activation = 'relu', padding = 'same')(x)
x = tf.keras.layers.BatchNormalization()(x)
x = tf.keras.layers.MaxPool2D()(x)
x = tf.keras.layers.Flatten()(x)
x = tflow.layers.MaskedDense(128, activation = 'relu')(x)
output_layer = tflow.layers.MaskedDense(10, activation = 'softmax')(x)

model = tf.keras.models.Model(input_layer, output_layer)

model.compile(loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'], optimizer = 'adam')
model = tflow.utils.mask_model(
    model,
    percentile = 75,
    x = x_train[:1000],
    y = y_train[:1000].reshape(-1, 1)
)
model.compile(loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'], optimizer = 'adam')

In [5]:
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 32, 32, 3)]       0         
                                                                 
 masked_conv2d (MaskedConv2D  (None, 32, 32, 16)       896       
 )                                                               
                                                                 
 batch_normalization (BatchN  (None, 32, 32, 16)       64        
 ormalization)                                                   
                                                                 
 masked_conv2d_1 (MaskedConv  (None, 32, 32, 16)       4640      
 2D)                                                             
                                                                 
 batch_normalization_1 (Batc  (None, 32, 32, 16)       64        
 hNormalization)                                             

In [ ]:
model.fit(
    x_train,
    y_train,
    validation_split = 0.2,
    batch_size = 256,
    epochs = 10,
    callbacks = tf.keras.callbacks.EarlyStopping(min_delta = 0.004, patience=2, restore_best_weights = True))

Epoch 1/10


2023-01-25 08:29:59.357870: W tensorflow/tsl/platform/profile_utils/cpu_utils.cc:128] Failed to get CPU frequency: 0 Hz


157/157 [==============================] - 43s 261ms/step - loss: 1.5812 - accuracy: 0.4273 - val_loss: 2.7896 - val_accuracy: 0.0997
Epoch 2/10
157/157 [==============================] - 41s 262ms/step - loss: 1.1800 - accuracy: 0.5784 - val_loss: 3.4930 - val_accuracy: 0.0952
Epoch 3/10
157/157 [==============================] - 42s 265ms/step - loss: 1.0113 - accuracy: 0.6413 - val_loss: 2.7168 - val_accuracy: 0.2871
Epoch 4/10
 41/157 [======>.......................] - ETA: 28s - loss: 0.9206 - accuracy: 0.6731

In [ ]:
preds = model.predict(x_test).argmax(axis = 1)
print(confusion_matrix(y_test, model.predict(x_test).argmax(axis = 1)))
print(classification_report(y_test, model.predict(x_test).argmax(axis = 1)))